# TCN-GNN-LSTM Hybrid Architecture

## A Complete Deep Learning Framework for Multi-Asset Crypto Portfolio Optimization

---

### Executive Summary

This document proposes and implements a **novel hybrid architecture** that combines three powerful deep learning paradigms:

1. **Temporal Convolutional Networks (TCN)** - For multi-scale temporal feature extraction
2. **Graph Neural Networks (GNN)** - For modeling dynamic cross-asset relationships
3. **Long Short-Term Memory (LSTM)** - For sequential prediction with memory

The architecture features a **multi-head output** system for:
- **Trading Head**: Direct portfolio weight predictions
- **Prediction Head**: Gaussian distribution (mean + uncertainty)
- **Value Head**: RL-compatible value function estimation

---

**Document Version:** 2.0 (Theory + Implementation)  
**Author:** AI Trading Research Team  
**Date:** February 2026  
**Status:** Complete Implementation with Theoretical Foundation

---

## Table of Contents

### Part I: Theoretical Foundation
1. [Motivation & Current Limitations](#1-motivation)
2. [Architecture Overview](#2-architecture)
3. [Component Deep Dive](#3-components)
   - 3.1 TCN Feature Extractor
   - 3.2 Dynamic GNN for Asset Correlations
   - 3.3 LSTM Sequential Processor
   - 3.4 Multi-Head Output System
4. [Mathematical Formulation](#4-math)
5. [Training Strategy - Curriculum Learning](#5-training-theory)
6. [Uncertainty-Aware Prediction](#6-uncertainty-theory)
7. [Loss Function Design](#7-loss-theory)

### Part II: Implementation
8. [Environment Setup](#8-setup)
9. [Data Loading Pipeline](#9-data)
10. [Exploratory Data Analysis](#10-eda)
11. [Feature Engineering](#11-features)
12. [Model Implementation](#12-model)
13. [Training Pipeline](#13-training)
14. [Evaluation & Results](#14-evaluation)
15. [Conclusion & Roadmap](#15-conclusion)

---

# PART I: THEORETICAL FOUNDATION

---

<a id="1-motivation"></a>
## 1. Motivation & Current System Limitations

### 1.1 Current Architecture

Our existing system uses:
- **Separate LSTM models** per asset for return prediction
- **Static correlation matrix** (computed once from historical data)
- **Mean-Variance Optimization** with predicted returns

### 1.2 Key Limitations

| Limitation | Impact | Proposed Solution |
|------------|--------|-------------------|
| **Single-scale features** | Misses multi-timeframe patterns | TCN with dilated convolutions |
| **Static correlations** | Fails during regime changes | Dynamic GNN updates |
| **No uncertainty quantification** | Overconfident predictions | Gaussian prediction head |
| **Single objective** | Trading vs prediction conflict | Multi-head architecture |
| **Sequential training only** | Suboptimal convergence | Curriculum learning |

### 1.3 Why This Architecture?

The proposed TCN-GNN-LSTM architecture addresses all limitations:

```
Current System:                    Proposed System:
+-----------+                      +------------------+
| LSTM only | Single-scale         | TCN              | Multi-scale
+-----------+                      | (1m, 5m, 1h, 1d) | temporal features
     |                             +------------------+
     v                                     |
+-----------+                              v
| Static    | Fixed correlations   +------------------+
| Corr Mat  |                      | GNN              | Dynamic, learned
+-----------+                      | (attention-based)| relationships
     |                             +------------------+
     v                                     |
+-----------+                              v
| MVO       | Point estimate only  +------------------+
| Optimizer |                      | LSTM + Multi-Head| Memory + 
+-----------+                      | (3 outputs)      | uncertainty
                                   +------------------+
```

---

<a id="2-architecture"></a>
## 2. Architecture Overview

### 2.1 High-Level Data Flow

```
                    RAW MARKET DATA
                    (OHLCV per asset)
                         |
                         v
    +--------------------------------------------+
    |              TCN FEATURE EXTRACTOR         |
    |                                            |
    |  Dilated Convolutions capture patterns at  |
    |  multiple time scales simultaneously       |
    |                                            |
    |  +--------+  +--------+  +--------+        |
    |  |Dilation|  |Dilation|  |Dilation|        |
    |  | d=1    |  | d=2    |  | d=4    |  ...   |
    |  | 1-day  |  | 2-day  |  | 4-day  |        |
    |  +--------+  +--------+  +--------+        |
    |       \          |           /             |
    |        \         |          /              |
    |         v        v         v               |
    |       [Multi-Scale Feature Vectors]        |
    +--------------------------------------------+
                         |
                         v
    +--------------------------------------------+
    |           GRAPH NEURAL NETWORK             |
    |                                            |
    |  Models assets as nodes in a graph         |
    |  Learns dynamic relationships via attention|
    |                                            |
    |    BTC ------- ETH                         |
    |     |  \     / |                           |
    |     |   \   /  |    Edge weights learned   |
    |     |    \ /   |    dynamically per batch  |
    |     |     X    |                           |
    |     |    / \   |                           |
    |     |   /   \  |                           |
    |    SOL ------ BNB                          |
    |                                            |
    |       [Cross-Asset Enriched Features]      |
    +--------------------------------------------+
                         |
                         v
    +--------------------------------------------+
    |              LSTM PROCESSOR                |
    |                                            |
    |  Bidirectional LSTM with temporal attention|
    |  Captures sequential dependencies and      |
    |  maintains memory of important events      |
    |                                            |
    |    h(t-2) --> h(t-1) --> h(t) --> ...     |
    |                                            |
    |       [Temporal Context + Memory]          |
    +--------------------------------------------+
                         |
           +-------------+-------------+
           |             |             |
           v             v             v
    +-----------+  +-----------+  +-----------+
    |  TRADING  |  | PREDICTION|  |   VALUE   |
    |   HEAD    |  |   HEAD    |  |   HEAD    |
    |           |  |           |  |           |
    | Outputs   |  | Outputs   |  | Outputs   |
    | portfolio |  | Gaussian  |  | expected  |
    | weights   |  | mu, sigma |  | return V  |
    | (softmax) |  | per asset |  | (scalar)  |
    +-----------+  +-----------+  +-----------+
           |             |             |
           v             v             v
    [Portfolio     [Uncertainty-  [Actor-Critic
     Allocation]    Aware Est.]    RL Training]
```

### 2.2 Tensor Shapes Throughout the Network

Understanding the data flow through each component:

| Stage | Shape | Description |
|-------|-------|-------------|
| **Input** | `(B, T, N, F)` | Batch, Time steps, N assets, F features |
| **Post-TCN** | `(B, T, N, D)` | D = TCN hidden dimension (e.g., 64) |
| **Post-GNN** | `(B, T, N, D)` | Same shape, enriched with cross-asset info |
| **Post-LSTM** | `(B, N, H)` | H = LSTM hidden (pooled over time) |
| **Trading Head** | `(B, N)` | Portfolio weights (sum to 1 via softmax) |
| **Prediction Head** | `(B, N, 2)` | Mean (μ) and std (σ) for each asset |
| **Value Head** | `(B, 1)` | Scalar value estimate for RL |

**Example dimensions:**
- `B` = 32 (batch size)
- `T` = 60 (60 days of history)
- `N` = 5 (5 crypto assets)
- `F` = 50 (50 input features per asset)
- `D` = 64 (TCN/GNN hidden dimension)
- `H` = 128 (LSTM hidden × 2 for bidirectional)

---

<a id="3-components"></a>
## 3. Component Deep Dive

### 3.1 Temporal Convolutional Network (TCN)

#### Why TCN Instead of Just LSTM?

Traditional LSTMs process data **sequentially**, which:
- Is slow (no parallelization possible)
- Has fixed receptive field
- Struggles with very long sequences (vanishing gradients)

**TCN solves this** using **dilated causal convolutions**:

#### How Dilated Convolutions Work

```
Standard Convolution (d=1):      Dilated Convolution (d=2):     Dilated Convolution (d=4):
                                                   
Output:   O O O O O O O O        Output:   O O O O O O O O      Output:   O O O O O O O O
          |\|\|\|\|\|\|\|                  | X | X | X | X                |   X   |   X   |
          | \| \| \| \| \|                 |/ \|/ \|/ \|/ \               |  / \  |  / \  |
Input:    I I I I I I I I        Input:    I I I I I I I I      Input:    I I I I I I I I
                                                   
Receptive Field: 2               Receptive Field: 4             Receptive Field: 8
Sees: today + yesterday          Sees: 4 days of patterns       Sees: 8 days of patterns
```

#### Key Insight: Exponential Receptive Field Growth

By stacking layers with dilation rates [1, 2, 4, 8, 16], we achieve:
- **Receptive field of 32 timesteps** with only 5 layers
- **Fully parallel computation** (unlike LSTM)
- **Multi-scale pattern capture** (short-term momentum + long-term trends)

#### Causal Property (Critical for Trading!)

```
CAUSAL (correct):               NON-CAUSAL (wrong - future leakage!):
                                
    Uses only PAST data             Uses FUTURE data (cheating!)
         ↓                                    ↓
    [..., t-2, t-1, t] → pred      [t-1, t, t+1, ...] → pred
                                              ↑
                                        Can't know this!
```

Our TCN uses `padding='causal'` to ensure predictions only use past information.

### 3.2 Graph Neural Network (GNN) for Cross-Asset Correlations

#### Why GNN?

Financial assets don't exist in isolation. Their relationships:
- Are **non-linear** (not captured by simple correlation coefficients)
- Are **time-varying** (correlations spike during market crashes)
- Have **asymmetric dependencies** (BTC leads, altcoins follow)

#### Traditional vs. GNN Approach

```
Traditional Correlation:           GNN-Based Relationships:
                                   
     Static Matrix                      Dynamic Graph
   +---+---+---+---+                    BTC
   |1.0|0.8|0.6|0.4|                   / | \
   +---+---+---+---+               0.9/  |  \0.7
   |0.8|1.0|0.7|0.5|                 /   |   \
   +---+---+---+---+              ETH    |   SOL
   |0.6|0.7|1.0|0.6|                 \   |   /
   +---+---+---+---+               0.8\  |  /0.6
   |0.4|0.5|0.6|1.0|                   \ | /
   +---+---+---+---+                    BNB
                                   
   Computed ONCE from history       Attention weights LEARNED
   Never changes                    and UPDATED every timestep
   Misses regime changes            Adapts to market conditions
```

#### How Graph Attention Works

For each pair of assets (i, j), we compute an attention score:

```
1. Project features:     Q_i = W_q * h_i    (query)
                         K_j = W_k * h_j    (key)
                         V_j = W_v * h_j    (value)

2. Compute attention:    attention(i,j) = softmax(Q_i · K_j / √d)

3. Aggregate:            h'_i = Σ_j attention(i,j) * V_j
```

**Result:** Each asset's representation is enriched with information from correlated assets, weighted by learned attention.

#### Multi-Head Attention

We use **4 attention heads** to capture different types of relationships:
- Head 1: Short-term momentum correlation
- Head 2: Volatility co-movement
- Head 3: Sector relationships (DeFi tokens, L1 chains, etc.)
- Head 4: Market-cap based hierarchy

### 3.3 LSTM Sequential Processor

After TCN extracts multi-scale features and GNN enriches them with cross-asset information, we use LSTM to:

1. **Capture longer temporal dependencies** beyond TCN's receptive field
2. **Maintain memory** of important events (crashes, halvings, etc.)
3. **Aggregate information** over time for final prediction

#### Bidirectional Processing

```
Forward LSTM:   h1 → h2 → h3 → h4 → h5 → h6
                 ↘   ↘   ↘   ↘   ↘   ↘
                      Concatenate
                 ↗   ↗   ↗   ↗   ↗   ↗
Backward LSTM:  h1 ← h2 ← h3 ← h4 ← h5 ← h6

Result: Each position has context from BOTH past and future
        (future here means "later in the sequence", not actual future)
```

#### Temporal Attention

Not all timesteps are equally important. We learn attention weights:

```
Timesteps:    [t-59, t-58, ..., t-2, t-1, t]
                ↓     ↓          ↓    ↓   ↓
Attention:   [0.01, 0.01, ..., 0.15, 0.20, 0.30]
                                      ↑
                            Recent data weighted more
                            (but not always - depends on patterns)
```

### 3.4 Multi-Head Output System

The key innovation of this architecture is the **three-headed output**:

```
                    LSTM Output
                  (batch, N, 256)
                        |
          +-------------+-------------+
          |             |             |
          v             v             v
   +-----------+  +-----------+  +-----------+
   |  TRADING  |  | PREDICTION|  |   VALUE   |
   |   HEAD    |  |   HEAD    |  |   HEAD    |
   +-----------+  +-----------+  +-----------+
   |           |  |           |  |           |
   | Dense(N)  |  | Dense(2N) |  | Dense(1)  |
   | Softmax   |  | μ, log(σ) |  | Linear    |
   |           |  |           |  |           |
   +-----------+  +-----------+  +-----------+
         |             |             |
         v             v             v
   [w_1,...,w_N] [μ_1,σ_1,...  [V(state)]
   Σw_i = 1       μ_N,σ_N]      scalar
```

#### Why Three Heads?

| Head | Purpose | Training Signal | Usage |
|------|---------|-----------------|-------|
| **Trading** | Direct portfolio weights | Sharpe ratio (primary loss) | Production decisions |
| **Prediction** | Return distribution | Gaussian NLL (auxiliary loss) | Uncertainty estimation |
| **Value** | Expected future reward | TD error (optional) | RL fine-tuning |

#### The Prediction Head: Why Gaussian?

Instead of predicting a single number, we predict a **distribution**:

```
Traditional:   "BTC will return +2.5% tomorrow"
                       ↓
               Single point estimate
               No confidence information

Our approach:  "BTC will return +2.5% ± 1.2% (68% confidence)"
                       ↓
               Mean (μ) = +2.5%
               Std (σ) = 1.2%
               We know HOW CONFIDENT we are!
```

---

<a id="4-math"></a>
## 4. Mathematical Formulation

### 4.1 Complete Model Forward Pass

Let's define the full forward pass mathematically:

**Input:** $X \in \mathbb{R}^{B \times T \times N \times F}$ (batch, time, assets, features)

**Step 1: TCN Feature Extraction**
$$Z_{tcn} = \text{TCN}(X) \in \mathbb{R}^{B \times T \times N \times D}$$

**Step 2: Graph Neural Network**
$$Z_{gnn}, A = \text{GNN}(Z_{tcn}) \in \mathbb{R}^{B \times T \times N \times D}, \mathbb{R}^{B \times T \times H \times N \times N}$$

Where $A$ is the attention matrix (interpretable correlations).

**Step 3: LSTM Processing**
$$H = \text{LSTM}(Z_{gnn}) \in \mathbb{R}^{B \times N \times 2D_{lstm}}$$

**Step 4: Multi-Head Outputs**
$$
\begin{aligned}
w &= \text{softmax}(\text{TradingHead}(\bar{H})) \in \Delta^{N-1} \text{ (simplex)}\\
\mu, \sigma &= \text{PredictionHead}(H) \in \mathbb{R}^{N} \times \mathbb{R}^{N}_{>0} \\
V &= \text{ValueHead}(\bar{H}) \in \mathbb{R}
\end{aligned}
$$

Where $\bar{H} = \frac{1}{N}\sum_i H_i$ is the mean-pooled representation.

### 4.2 Dilated Convolution Formula

A dilated convolution with dilation rate $d$ is defined as:

$$(x *_d k)(t) = \sum_{i=0}^{K-1} k(i) \cdot x(t - d \cdot i)$$

Where:
- $x$ is the input sequence
- $k$ is the kernel of size $K$
- $d$ is the dilation rate

**Receptive Field Calculation:**

For TCN with $L$ layers and dilations $[1, 2, 4, ..., 2^{L-1}]$:

$$\text{Receptive Field} = 1 + (K-1) \cdot \sum_{i=0}^{L-1} 2^i = 1 + (K-1) \cdot (2^L - 1)$$

For $K=3$ and $L=4$: RF = $1 + 2 \cdot 15 = 31$ timesteps

### 4.3 Graph Attention Formula

For nodes $i$ and $j$ at time $t$:

**Attention Score:**
$$e_{ij}^{(t)} = \frac{(W_Q h_i^{(t)})^T (W_K h_j^{(t)})}{\sqrt{d_k}}$$

**Normalized Attention:**
$$\alpha_{ij}^{(t)} = \frac{\exp(e_{ij}^{(t)})}{\sum_{k=1}^{N} \exp(e_{ik}^{(t)})}$$

**Updated Features:**
$$h_i'^{(t)} = \sum_{j=1}^{N} \alpha_{ij}^{(t)} (W_V h_j^{(t)})$$

**Key Insight:** Unlike static correlation, $\alpha_{ij}^{(t)}$ is computed **fresh for each timestep**, capturing time-varying relationships.

---

<a id="5-training-theory"></a>
## 5. Training Strategy: Curriculum Learning

### 5.1 Why Curriculum Learning?

Training a complex multi-head model end-to-end from scratch often fails because:
- Gradients from different heads **conflict**
- The model hasn't learned good **representations** yet
- Trading objectives are **noisy** (reward depends on market randomness)

**Curriculum Learning** solves this by training in stages, from simple to complex:

### 5.2 Three-Stage Training

```
╔══════════════════════════════════════════════════════════════════════╗
║  STAGE 1: REPRESENTATION LEARNING                                    ║
╠══════════════════════════════════════════════════════════════════════╣
║  Goal:    Learn good features from data                              ║
║  Train:   Prediction head ONLY (Gaussian NLL loss)                   ║
║  Freeze:  Trading head, Value head                                   ║
║  Why:     Supervised learning is stable, provides clean gradients    ║
║  Loss:    λ_pred = 1.0, λ_trading = 0.0, λ_value = 0.0              ║
╚══════════════════════════════════════════════════════════════════════╝
                                ↓
╔══════════════════════════════════════════════════════════════════════╗
║  STAGE 2: TRADING OBJECTIVE FINE-TUNING                              ║
╠══════════════════════════════════════════════════════════════════════╣
║  Goal:    Optimize for actual trading performance                    ║
║  Train:   Trading head (Sharpe loss) + Prediction head (regularizer) ║
║  Freeze:  Value head                                                 ║
║  Why:     Now that representations are good, fine-tune for trading   ║
║  Loss:    λ_pred = 0.1, λ_trading = 1.0, λ_value = 0.0              ║
║  Note:    Prediction head prevents overfitting to noise              ║
╚══════════════════════════════════════════════════════════════════════╝
                                ↓
╔══════════════════════════════════════════════════════════════════════╗
║  STAGE 3: RL ENHANCEMENT (Optional)                                  ║
╠══════════════════════════════════════════════════════════════════════╣
║  Goal:    Learn from actual trading simulation                       ║
║  Train:   All heads with RL (PPO/A2C)                                ║
║  Why:     RL can discover strategies that supervised learning misses ║
║  Loss:    λ_pred = 0.05, λ_trading = 0.5, λ_value = 0.45            ║
║  Caution: RL is unstable, use sparingly                              ║
╚══════════════════════════════════════════════════════════════════════╝
```

---

<a id="6-uncertainty-theory"></a>
## 6. Uncertainty-Aware Prediction

### 6.1 The Problem with Point Estimates

Traditional models output a single prediction:
```
Model says: "BTC will return +2.5% tomorrow"

But is it:  +2.5% ± 0.5%  (HIGH confidence - trade aggressively!)
    or:     +2.5% ± 5.0%  (LOW confidence - maybe don't trade...)

Point estimates DON'T TELL US!
```

### 6.2 Gaussian Output: Mean + Uncertainty

Our prediction head outputs TWO values per asset:
- **μ (mean)**: Expected return
- **σ (sigma)**: Uncertainty/confidence

### 6.3 Uncertainty-Adjusted Portfolio Weights

The key insight: **reduce allocation to uncertain predictions**

```
╔════════════════════════════════════════════════════════════════════╗
║  TRADITIONAL APPROACH                                              ║
╠════════════════════════════════════════════════════════════════════╣
║  Model Output:        Raw Weights:                                 ║
║    BTC: +3%             BTC: 30%                                   ║
║    ETH: +4%      →      ETH: 40%                                   ║
║    SOL: +2%             SOL: 30%                                   ║
║                                                                    ║
║  Problem: What if ETH prediction is very uncertain?                ║
╚════════════════════════════════════════════════════════════════════╝

╔════════════════════════════════════════════════════════════════════╗
║  OUR UNCERTAINTY-AWARE APPROACH                                    ║
╠════════════════════════════════════════════════════════════════════╣
║  Model Output:                    Adjusted Weights:                ║
║    BTC: +3% (σ=1%)  HIGH conf       BTC: 45%  ↑                   ║
║    ETH: +4% (σ=5%)  LOW conf  →     ETH: 25%  ↓ (penalized!)      ║
║    SOL: +2% (σ=2%)  MED conf        SOL: 30%                      ║
║                                                                    ║
║  Formula: w_adjusted = w_raw × exp(-penalty × σ)                   ║
║           then normalize to sum to 1                               ║
╚════════════════════════════════════════════════════════════════════╝
```

### 6.4 When NOT to Trade

If average confidence is below a threshold, we can **skip trading**:

```python
if avg_confidence < 0.5:
    action = "HOLD - Model is too uncertain"
else:
    action = "TRADE with uncertainty-adjusted weights"
```

---

<a id="7-loss-theory"></a>
## 7. Loss Function Design

### 7.1 Multi-Objective Loss

The total loss is a weighted combination:

$$\mathcal{L}_{total} = \lambda_1 \mathcal{L}_{trading} + \lambda_2 \mathcal{L}_{prediction} + \lambda_3 \mathcal{L}_{value}$$

### 7.2 Trading Loss: Negative Sharpe Ratio

$$\mathcal{L}_{trading} = -\text{Sharpe}(r_p) = -\frac{\mathbb{E}[r_p]}{\text{Std}[r_p]} \times \sqrt{252}$$

Where $r_p = \sum_i w_i \cdot r_i$ is the portfolio return.

**Why Sharpe?** We want to maximize **risk-adjusted** returns, not just returns.

### 7.3 Prediction Loss: Gaussian Negative Log-Likelihood

$$\mathcal{L}_{prediction} = \frac{1}{2} \left[ \log(\sigma^2) + \frac{(y - \mu)^2}{\sigma^2} \right]$$

**Key Properties:**
```
If prediction is CORRECT (y ≈ μ):
  - Small σ → Small loss  (rewarded for confidence!)
  - Large σ → Medium loss (unnecessary uncertainty)

If prediction is WRONG (y ≠ μ):
  - Small σ → LARGE loss  (punished for overconfidence!)
  - Large σ → Medium loss (at least we knew we were uncertain)
```

This naturally teaches the model to be:
- **Confident** when it can be accurate
- **Uncertain** when predictions are unreliable

### 7.4 Why Prediction Loss as Regularizer?

Using prediction loss (with small weight λ₂=0.1) during trading training:
- **Prevents overfitting** to spurious trading patterns
- **Forces backbone** to learn generalizable features
- Acts like **auxiliary task** in multi-task learning

---

# PART II: IMPLEMENTATION

---

<a id="8-setup"></a>
## 8. Environment Setup

In [ ]:
# ============================================
# INSTALL DEPENDENCIES (Run this first in Colab)
# ============================================
# !pip install ccxt ta tensorflow pandas numpy matplotlib seaborn scikit-learn -q

In [ ]:
# ============================================
# IMPORTS
# ============================================
import numpy as np
import pandas as pd
from datetime import datetime, timezone, timedelta
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = (14, 6)

# Data fetching
import ccxt

# Technical analysis
import ta

# Machine Learning
from sklearn.preprocessing import RobustScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model

# Utilities
from typing import Dict, List, Tuple
import time
from scipy import stats

print("="*70)
print("TCN-GNN-LSTM HYBRID ARCHITECTURE")
print("="*70)
print(f"TensorFlow: {tf.__version__}")
print(f"NumPy:      {np.__version__}")
print(f"Pandas:     {pd.__version__}")
print(f"CCXT:       {ccxt.__version__}")
print("="*70)

<a id="9-data"></a>
## 9. Data Loading Pipeline

In [ ]:
class CryptoDataLoader:
    """
    Data loader for cryptocurrency OHLCV data.
    
    Uses CCXT library for unified exchange API access.
    Supports chunked fetching for large date ranges.
    """
    
    def __init__(self, exchange_name='binance'):
        self.exchange = self._init_exchange(exchange_name)
        self.cache = {}
        
    def _init_exchange(self, name):
        """Initialize exchange connection."""
        exchanges = {
            'binance': ccxt.binance,
            'coinbase': ccxt.coinbase,
            'kraken': ccxt.kraken,
        }
        exchange_class = exchanges.get(name, ccxt.binance)
        return exchange_class({'enableRateLimit': True})
    
    def fetch_ohlcv(self, symbol: str, timeframe: str = '1d', 
                    limit: int = 365) -> pd.DataFrame:
        """
        Fetch OHLCV data for a symbol.
        
        Args:
            symbol: Trading pair (e.g., 'BTC/USDT')
            timeframe: Candle timeframe ('1d', '4h', '1h')
            limit: Number of candles to fetch
        """
        cache_key = f"{symbol}_{timeframe}_{limit}"
        if cache_key in self.cache:
            return self.cache[cache_key]
        
        try:
            data = self.exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
            df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True)
            df.set_index('timestamp', inplace=True)
            self.cache[cache_key] = df
            return df
        except Exception as e:
            print(f"Error fetching {symbol}: {e}")
            return pd.DataFrame()
    
    def fetch_multi_asset(self, symbols: List[str], timeframe: str = '1d',
                          limit: int = 365) -> Dict[str, pd.DataFrame]:
        """Fetch data for multiple assets."""
        data = {}
        for symbol in symbols:
            print(f"  Fetching {symbol}...", end=" ")
            df = self.fetch_ohlcv(symbol, timeframe, limit)
            if not df.empty:
                data[symbol] = df
                print(f"✓ {len(df)} candles")
            else:
                print("✗ Failed")
            time.sleep(0.5)  # Rate limiting
        return data


# Initialize loader
loader = CryptoDataLoader('binance')
print(f"\nExchange: {loader.exchange.name}")

In [ ]:
# ============================================
# PORTFOLIO CONFIGURATION
# ============================================

PORTFOLIO_ASSETS = [
    "BTC/USDT",   # Bitcoin
    "ETH/USDT",   # Ethereum
    "BNB/USDT",   # Binance Coin
    "SOL/USDT",   # Solana
    "XRP/USDT",   # Ripple
]

TIMEFRAME = '1d'
LOOKBACK_DAYS = 365

print(f"Assets: {len(PORTFOLIO_ASSETS)}")
print(f"Timeframe: {TIMEFRAME}")
print(f"Lookback: {LOOKBACK_DAYS} days")

In [ ]:
# ============================================
# FETCH DATA
# ============================================

print("\nFetching OHLCV data...")
raw_data = loader.fetch_multi_asset(PORTFOLIO_ASSETS, TIMEFRAME, LOOKBACK_DAYS)
print(f"\n✓ Loaded {len(raw_data)} assets")

<a id="10-eda"></a>
## 10. Exploratory Data Analysis

In [ ]:
# ============================================
# PRICE VISUALIZATION
# ============================================

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()
colors = ['#F7931A', '#627EEA', '#F3BA2F', '#00FFA3', '#23292F']

for idx, (symbol, df) in enumerate(raw_data.items()):
    if idx >= 5:
        break
    ax = axes[idx]
    ax.plot(df.index, df['close'], color=colors[idx], linewidth=1.5)
    ax.fill_between(df.index, df['low'], df['high'], alpha=0.2, color=colors[idx])
    ax.set_title(f"{symbol.replace('/USDT', '')} Price", fontweight='bold')
    ax.tick_params(axis='x', rotation=45)

axes[5].set_visible(False)
plt.suptitle('Asset Price History', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# ============================================
# CORRELATION ANALYSIS (Key insight for GNN!)
# ============================================

# Calculate returns
returns_data = {s.replace('/USDT', ''): df['close'].pct_change().dropna() 
                for s, df in raw_data.items()}
returns_df = pd.DataFrame(returns_data)

# Correlation matrix
corr_matrix = returns_df.corr()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Static correlation
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='RdYlGn', 
            center=0, ax=axes[0], vmin=-1, vmax=1)
axes[0].set_title('Static Correlation Matrix', fontweight='bold')

# Rolling correlation (shows time-varying nature)
for col in returns_df.columns[1:]:
    rolling_corr = returns_df['BTC'].rolling(30).corr(returns_df[col])
    axes[1].plot(rolling_corr, label=f'BTC-{col}')

axes[1].axhline(y=0, color='white', linestyle='--', alpha=0.3)
axes[1].set_title('30-Day Rolling Correlation with BTC', fontweight='bold')
axes[1].legend()
axes[1].set_ylim(-1, 1)

plt.tight_layout()
plt.show()

print("\n📊 Key Insight: Correlations vary over time!")
print("   → This is why we need a GNN with dynamic edge weights")

<a id="11-features"></a>
## 11. Feature Engineering

In [ ]:
def add_technical_indicators(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add comprehensive technical indicators.
    
    Categories:
    - Trend (MA, EMA, MACD)
    - Momentum (RSI, Stochastic)
    - Volatility (Bollinger, ATR)
    - Volume (OBV, MFI)
    """
    df = df.copy()
    
    # Moving Averages
    for period in [7, 14, 21, 50]:
        df[f'sma_{period}'] = df['close'].rolling(window=period).mean()
        df[f'ema_{period}'] = df['close'].ewm(span=period, adjust=False).mean()
    
    # MACD
    macd = ta.trend.MACD(df['close'])
    df['macd'] = macd.macd()
    df['macd_signal'] = macd.macd_signal()
    df['macd_diff'] = macd.macd_diff()
    
    # RSI
    for period in [7, 14, 21]:
        df[f'rsi_{period}'] = ta.momentum.RSIIndicator(df['close'], window=period).rsi()
    
    # Stochastic
    stoch = ta.momentum.StochasticOscillator(df['high'], df['low'], df['close'])
    df['stoch_k'] = stoch.stoch()
    df['stoch_d'] = stoch.stoch_signal()
    
    # Bollinger Bands
    bb = ta.volatility.BollingerBands(df['close'])
    df['bb_high'] = bb.bollinger_hband()
    df['bb_low'] = bb.bollinger_lband()
    df['bb_width'] = bb.bollinger_wband()
    
    # ATR
    df['atr_14'] = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close']).average_true_range()
    
    # Volume indicators
    df['obv'] = ta.volume.OnBalanceVolumeIndicator(df['close'], df['volume']).on_balance_volume()
    df['mfi'] = ta.volume.MFIIndicator(df['high'], df['low'], df['close'], df['volume']).money_flow_index()
    
    # Returns
    for period in [1, 3, 5, 10]:
        df[f'return_{period}d'] = df['close'].pct_change(period)
    
    return df


# Apply to all assets
print("\nApplying technical indicators...")
featured_data = {}
for symbol, df in raw_data.items():
    featured_data[symbol] = add_technical_indicators(df)
    print(f"  {symbol}: {len(featured_data[symbol].columns)} features")

print(f"\n✓ Feature engineering complete")

<a id="12-model"></a>
## 12. Model Implementation

### 12.1 Data Preprocessing

In [ ]:
class DataPreprocessor:
    """Preprocess multi-asset data for the model."""
    
    def __init__(self):
        self.scalers = {}
        
    def prepare_data(self, data: Dict[str, pd.DataFrame], 
                     target_col: str = 'return_1d') -> Tuple[np.ndarray, np.ndarray, List[str]]:
        """Prepare multi-asset data arrays."""
        # Find common dates
        common_index = None
        for df in data.values():
            idx = df.index
            common_index = idx if common_index is None else common_index.intersection(idx)
        
        asset_names = [s.replace('/USDT', '') for s in data.keys()]
        
        # Get feature columns
        exclude = ['open', 'high', 'low', 'close', 'volume']
        feature_cols = [c for c in list(data.values())[0].columns if c not in exclude]
        
        X_list, y_list = [], []
        
        for symbol, df in data.items():
            df_aligned = df.loc[common_index]
            
            # Scale features
            scaler = RobustScaler()
            features = df_aligned[feature_cols].values
            features = np.nan_to_num(features, nan=0.0)
            features = scaler.fit_transform(features)
            self.scalers[symbol] = scaler
            
            X_list.append(features)
            y_list.append(df_aligned[target_col].shift(-1).values)
        
        X = np.stack(X_list, axis=1)  # (time, assets, features)
        y = np.stack(y_list, axis=1)  # (time, assets)
        y = np.nan_to_num(y, nan=0.0)
        
        return X[:-1], y[:-1], asset_names
    
    def create_sequences(self, X: np.ndarray, y: np.ndarray, 
                         seq_length: int = 60) -> Tuple[np.ndarray, np.ndarray]:
        """Create sequences for temporal models."""
        X_seq = [X[i-seq_length:i] for i in range(seq_length, len(X))]
        y_seq = [y[i] for i in range(seq_length, len(y))]
        return np.array(X_seq), np.array(y_seq)


# Preprocess data
print("\nPreprocessing data...")
preprocessor = DataPreprocessor()
X, y, asset_names = preprocessor.prepare_data(featured_data)
print(f"  X shape: {X.shape}")
print(f"  y shape: {y.shape}")

SEQ_LENGTH = 60
X_seq, y_seq = preprocessor.create_sequences(X, y, SEQ_LENGTH)
print(f"  X_seq shape: {X_seq.shape}")
print(f"  y_seq shape: {y_seq.shape}")

# Split data
n = len(X_seq)
train_end = int(n * 0.7)
val_end = int(n * 0.85)

X_train, y_train = X_seq[:train_end], y_seq[:train_end]
X_val, y_val = X_seq[train_end:val_end], y_seq[train_end:val_end]
X_test, y_test = X_seq[val_end:], y_seq[val_end:]

print(f"\n  Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

### 12.2 TCN Block

In [ ]:
class TCNBlock(layers.Layer):
    """
    Temporal Convolutional Block with:
    - Dilated causal convolution
    - Residual connection
    - Layer normalization
    """
    
    def __init__(self, filters, kernel_size=3, dilation_rate=1, dropout=0.2, **kwargs):
        super().__init__(**kwargs)
        self.conv1 = layers.Conv1D(filters, kernel_size, dilation_rate=dilation_rate, 
                                    padding='causal', kernel_initializer='he_normal')
        self.conv2 = layers.Conv1D(filters, kernel_size, dilation_rate=dilation_rate,
                                    padding='causal', kernel_initializer='he_normal')
        self.norm1 = layers.LayerNormalization()
        self.norm2 = layers.LayerNormalization()
        self.dropout1 = layers.Dropout(dropout)
        self.dropout2 = layers.Dropout(dropout)
        self.residual = layers.Conv1D(filters, 1)
        
    def call(self, x, training=False):
        out = self.conv1(x)
        out = self.norm1(out)
        out = tf.nn.gelu(out)
        out = self.dropout1(out, training=training)
        
        out = self.conv2(out)
        out = self.norm2(out)
        out = tf.nn.gelu(out)
        out = self.dropout2(out, training=training)
        
        return out + self.residual(x)


class TCNFeatureExtractor(layers.Layer):
    """Multi-scale TCN with exponential dilations."""
    
    def __init__(self, channels=64, num_layers=4, dropout=0.2, **kwargs):
        super().__init__(**kwargs)
        self.blocks = [TCNBlock(channels, dilation_rate=2**i, dropout=dropout)
                       for i in range(num_layers)]
        
    def call(self, x, training=False):
        for block in self.blocks:
            x = block(x, training=training)
        return x


print("✓ TCN layers defined")

### 12.3 Graph Attention Layer

In [ ]:
class GraphAttentionLayer(layers.Layer):
    """
    Graph Attention for dynamic cross-asset relationships.
    
    Computes attention weights between all asset pairs,
    allowing the model to learn time-varying correlations.
    """
    
    def __init__(self, hidden_dim, num_heads=4, dropout=0.1, **kwargs):
        super().__init__(**kwargs)
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        self.head_dim = hidden_dim // num_heads
        
        self.query = layers.Dense(hidden_dim)
        self.key = layers.Dense(hidden_dim)
        self.value = layers.Dense(hidden_dim)
        self.output_proj = layers.Dense(hidden_dim)
        self.dropout = layers.Dropout(dropout)
        self.norm = layers.LayerNormalization()
        
    def call(self, x, training=False):
        batch, time, assets, features = tf.shape(x)[0], tf.shape(x)[1], tf.shape(x)[2], tf.shape(x)[3]
        
        # Reshape: (batch * time, assets, features)
        x_flat = tf.reshape(x, [-1, assets, self.hidden_dim])
        
        # Compute Q, K, V
        Q = self.query(x_flat)
        K = self.key(x_flat)
        V = self.value(x_flat)
        
        # Multi-head reshape
        Q = tf.reshape(Q, [-1, assets, self.num_heads, self.head_dim])
        K = tf.reshape(K, [-1, assets, self.num_heads, self.head_dim])
        V = tf.reshape(V, [-1, assets, self.num_heads, self.head_dim])
        
        Q = tf.transpose(Q, [0, 2, 1, 3])
        K = tf.transpose(K, [0, 2, 1, 3])
        V = tf.transpose(V, [0, 2, 1, 3])
        
        # Attention
        scores = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(tf.cast(self.head_dim, tf.float32))
        attn = tf.nn.softmax(scores, axis=-1)
        attn = self.dropout(attn, training=training)
        
        out = tf.matmul(attn, V)
        out = tf.transpose(out, [0, 2, 1, 3])
        out = tf.reshape(out, [-1, assets, self.hidden_dim])
        
        out = self.output_proj(out)
        out = self.norm(out + x_flat)
        
        return tf.reshape(out, [batch, time, assets, self.hidden_dim])


print("✓ Graph Attention layer defined")

### 12.4 LSTM Processor

In [ ]:
class LSTMProcessor(layers.Layer):
    """
    Bidirectional LSTM with temporal attention.
    """
    
    def __init__(self, hidden_dim=64, num_layers=2, dropout=0.2, **kwargs):
        super().__init__(**kwargs)
        self.lstm_layers = [layers.Bidirectional(layers.LSTM(hidden_dim, return_sequences=True, dropout=dropout))
                            for _ in range(num_layers)]
        self.attention = layers.Dense(1, activation='tanh')
        self.norm = layers.LayerNormalization()
        
    def call(self, x, training=False):
        batch, time, assets, features = tf.shape(x)[0], tf.shape(x)[1], tf.shape(x)[2], tf.shape(x)[3]
        
        # Process each asset
        x = tf.transpose(x, [0, 2, 1, 3])  # (batch, assets, time, features)
        x = tf.reshape(x, [-1, time, features])
        
        for lstm in self.lstm_layers:
            x = lstm(x, training=training)
        
        # Temporal attention
        attn = tf.nn.softmax(self.attention(x), axis=1)
        x = tf.reduce_sum(x * attn, axis=1)
        
        x = tf.reshape(x, [batch, assets, -1])
        return self.norm(x)


print("✓ LSTM Processor defined")

### 12.5 Multi-Head Output

In [ ]:
class MultiHeadOutput(layers.Layer):
    """
    Three output heads:
    1. Trading: Portfolio weights (softmax)
    2. Prediction: Gaussian parameters (μ, σ)
    3. Value: Scalar value estimate
    """
    
    def __init__(self, num_assets, hidden_dim=64, **kwargs):
        super().__init__(**kwargs)
        self.num_assets = num_assets
        
        # Trading head
        self.trading_dense = layers.Dense(hidden_dim, activation='relu')
        self.trading_out = layers.Dense(num_assets)
        
        # Prediction head
        self.pred_dense = layers.Dense(hidden_dim, activation='relu')
        self.pred_mu = layers.Dense(num_assets)
        self.pred_sigma = layers.Dense(num_assets)
        
        # Value head
        self.value_dense = layers.Dense(hidden_dim, activation='relu')
        self.value_out = layers.Dense(1)
        
    def call(self, x, training=False):
        # Global pooling
        global_feat = tf.reduce_mean(x, axis=1)
        
        # Trading
        trading = self.trading_dense(global_feat)
        weights = tf.nn.softmax(self.trading_out(trading), axis=-1)
        
        # Prediction
        pred = self.pred_dense(x)
        mu = tf.reduce_mean(self.pred_mu(pred), axis=-1)
        log_sigma = tf.reduce_mean(self.pred_sigma(pred), axis=-1)
        log_sigma = tf.clip_by_value(log_sigma, -4.6, 2.3)
        sigma = tf.exp(log_sigma)
        
        # Value
        value = self.value_dense(global_feat)
        value = self.value_out(value)
        
        return weights, mu, sigma, value


print("✓ Multi-Head Output defined")

### 12.6 Complete Model

In [ ]:
class TCN_GNN_LSTM_Model(Model):
    """
    Complete TCN-GNN-LSTM Hybrid Model.
    
    Architecture:
    Input → TCN → GNN → LSTM → Multi-Head Output
    """
    
    def __init__(self, num_assets, input_features, 
                 tcn_channels=32, gnn_heads=2, lstm_hidden=32, **kwargs):
        super().__init__(**kwargs)
        
        self.num_assets = num_assets
        self.input_features = input_features
        
        self.input_proj = layers.Dense(tcn_channels, activation='relu')
        self.tcn = TCNFeatureExtractor(tcn_channels, num_layers=3)
        self.gnn = GraphAttentionLayer(tcn_channels, num_heads=gnn_heads)
        self.lstm = LSTMProcessor(lstm_hidden, num_layers=1)
        self.output_heads = MultiHeadOutput(num_assets, lstm_hidden * 2)
        
    def call(self, x, training=False):
        batch = tf.shape(x)[0]
        time = tf.shape(x)[1]
        assets = tf.shape(x)[2]
        
        # TCN per asset
        x = tf.transpose(x, [0, 2, 1, 3])
        x = tf.reshape(x, [-1, time, self.input_features])
        x = self.input_proj(x)
        x = self.tcn(x, training=training)
        x = tf.reshape(x, [batch, assets, time, -1])
        x = tf.transpose(x, [0, 2, 1, 3])
        
        # GNN
        x = self.gnn(x, training=training)
        
        # LSTM
        x = self.lstm(x, training=training)
        
        # Output
        return self.output_heads(x, training=training)


# Build model
num_assets = len(PORTFOLIO_ASSETS)
num_features = X_seq.shape[-1]

model = TCN_GNN_LSTM_Model(
    num_assets=num_assets,
    input_features=num_features,
    tcn_channels=32,
    gnn_heads=2,
    lstm_hidden=32
)

# Test forward pass
test_input = tf.random.normal((4, SEQ_LENGTH, num_assets, num_features))
w, mu, sigma, v = model(test_input)

print("\n" + "="*60)
print("MODEL BUILT SUCCESSFULLY")
print("="*60)
print(f"  Input:   {test_input.shape}")
print(f"  Weights: {w.shape}")
print(f"  Mu:      {mu.shape}")
print(f"  Sigma:   {sigma.shape}")
print(f"  Value:   {v.shape}")

<a id="13-training"></a>
## 13. Training Pipeline

In [ ]:
# ============================================
# LOSS FUNCTIONS
# ============================================

def sharpe_loss(y_true, weights, epsilon=1e-8):
    """Negative Sharpe ratio loss."""
    portfolio_ret = tf.reduce_sum(weights * y_true, axis=-1)
    mean_ret = tf.reduce_mean(portfolio_ret)
    std_ret = tf.math.reduce_std(portfolio_ret) + epsilon
    return -mean_ret / std_ret * tf.math.sqrt(252.0)

def gaussian_nll(y_true, mu, sigma, epsilon=1e-8):
    """Gaussian negative log-likelihood."""
    sigma = tf.maximum(sigma, epsilon)
    nll = 0.5 * (tf.math.log(sigma**2) + (y_true - mu)**2 / sigma**2)
    return tf.reduce_mean(nll)

class MultiTaskLoss:
    """Combined loss with curriculum learning weights."""
    def __init__(self, lambda_trading=1.0, lambda_pred=0.1):
        self.lambda_trading = lambda_trading
        self.lambda_pred = lambda_pred
        
    def __call__(self, y_true, weights, mu, sigma):
        l_trading = sharpe_loss(y_true, weights) if self.lambda_trading > 0 else 0
        l_pred = gaussian_nll(y_true, mu, sigma) if self.lambda_pred > 0 else 0
        return self.lambda_trading * l_trading + self.lambda_pred * l_pred

print("✓ Loss functions defined")

In [ ]:
# ============================================
# TRAINING
# ============================================

optimizer = keras.optimizers.Adam(learning_rate=1e-3)

# Stage 1: Representation learning
print("\n--- Stage 1: Representation Learning ---")
loss_fn = MultiTaskLoss(lambda_trading=0.0, lambda_pred=1.0)

for epoch in range(10):
    losses = []
    for i in range(0, len(X_train) - 16, 16):
        X_batch = tf.constant(X_train[i:i+16], dtype=tf.float32)
        y_batch = tf.constant(y_train[i:i+16], dtype=tf.float32)
        
        with tf.GradientTape() as tape:
            w, mu, sigma, v = model(X_batch, training=True)
            loss = loss_fn(y_batch, w, mu, sigma)
        
        grads = tape.gradient(loss, model.trainable_variables)
        grads, _ = tf.clip_by_global_norm(grads, 1.0)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        losses.append(loss.numpy())
    
    print(f"  Epoch {epoch+1}: Loss = {np.mean(losses):.4f}")

# Stage 2: Trading fine-tuning
print("\n--- Stage 2: Trading Fine-tuning ---")
loss_fn = MultiTaskLoss(lambda_trading=1.0, lambda_pred=0.1)
optimizer.learning_rate.assign(5e-4)

for epoch in range(10):
    losses = []
    for i in range(0, len(X_train) - 16, 16):
        X_batch = tf.constant(X_train[i:i+16], dtype=tf.float32)
        y_batch = tf.constant(y_train[i:i+16], dtype=tf.float32)
        
        with tf.GradientTape() as tape:
            w, mu, sigma, v = model(X_batch, training=True)
            loss = loss_fn(y_batch, w, mu, sigma)
        
        grads = tape.gradient(loss, model.trainable_variables)
        grads, _ = tf.clip_by_global_norm(grads, 1.0)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        losses.append(loss.numpy())
    
    print(f"  Epoch {epoch+1}: Loss = {np.mean(losses):.4f}")

print("\n✓ Training complete!")

<a id="14-evaluation"></a>
## 14. Evaluation & Results

In [ ]:
# ============================================
# EVALUATION
# ============================================

X_test_tensor = tf.constant(X_test, dtype=tf.float32)
weights, mu, sigma, value = model(X_test_tensor, training=False)
weights = weights.numpy()
mu = mu.numpy()
sigma = sigma.numpy()

# Portfolio returns
portfolio_returns = np.sum(weights * y_test, axis=1)

# Metrics
total_return = (1 + portfolio_returns).prod() - 1
sharpe = portfolio_returns.mean() / portfolio_returns.std() * np.sqrt(252)

# Max drawdown
cumulative = (1 + portfolio_returns).cumprod()
running_max = np.maximum.accumulate(cumulative)
max_dd = ((cumulative - running_max) / running_max).min()

print("\n" + "="*60)
print("TEST RESULTS")
print("="*60)
print(f"  Total Return:  {total_return*100:.2f}%")
print(f"  Sharpe Ratio:  {sharpe:.3f}")
print(f"  Max Drawdown:  {max_dd*100:.2f}%")

In [ ]:
# ============================================
# VISUALIZATIONS
# ============================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Portfolio allocation
ax = axes[0, 0]
avg_weights = weights.mean(axis=0)
ax.pie(avg_weights, labels=asset_names, autopct='%1.1f%%', colors=colors)
ax.set_title('Average Portfolio Allocation', fontweight='bold')

# 2. Equity curve
ax = axes[0, 1]
ax.plot((1 + portfolio_returns).cumprod(), label='TCN-GNN-LSTM', color='cyan', linewidth=2)
equal_ret = np.sum(np.ones_like(weights) / num_assets * y_test, axis=1)
ax.plot((1 + equal_ret).cumprod(), label='Equal Weight', color='orange', linestyle='--')
ax.axhline(y=1, color='white', linestyle=':', alpha=0.3)
ax.set_title('Cumulative Returns', fontweight='bold')
ax.legend()

# 3. Uncertainty over time
ax = axes[1, 0]
for i, name in enumerate(asset_names):
    ax.plot(sigma[:, i], label=name, alpha=0.7)
ax.set_title('Prediction Uncertainty (σ)', fontweight='bold')
ax.legend()

# 4. Weights over time
ax = axes[1, 1]
ax.stackplot(range(len(weights)), weights.T, labels=asset_names, colors=colors, alpha=0.8)
ax.set_title('Portfolio Weights Over Time', fontweight='bold')
ax.legend(loc='upper right')
ax.set_ylim(0, 1)

plt.tight_layout()
plt.show()

<a id="15-conclusion"></a>
## 15. Conclusion & Roadmap

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════════╗
║               TCN-GNN-LSTM ARCHITECTURE SUMMARY                      ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                      ║
║  COMPONENTS IMPLEMENTED:                                             ║
║  ✓ TCN Feature Extractor (multi-scale patterns)                      ║
║  ✓ Graph Neural Network (dynamic correlations)                       ║
║  ✓ LSTM Processor (temporal memory)                                  ║
║  ✓ Multi-Head Output (trading + prediction + value)                  ║
║                                                                      ║
║  KEY INNOVATIONS:                                                    ║
║  • Uncertainty quantification via Gaussian head                      ║
║  • Curriculum learning (3 stages)                                    ║
║  • Dynamic correlation modeling via attention                        ║
║  • Multi-task regularization                                         ║
║                                                                      ║
║  NEXT STEPS:                                                         ║
║  1. Scale to 20+ assets                                              ║
║  2. Add Stage 3 (RL enhancement)                                     ║
║  3. Implement real-time inference                                    ║
║  4. Production deployment                                            ║
║                                                                      ║
╚══════════════════════════════════════════════════════════════════════╝
""")